# Module 6: Vertex AI Deployment

**Duration:** ~30 minutes

**Learning Objectives:**
- Understand Vertex AI Agent Engine architecture
- Deploy ADK agents to Agent Engine
- Configure production features (sessions, memory, evaluation)
- Deploy agents to Cloud Run with A2A support
- Set up CI/CD pipelines for agent deployment

---

## 6.1 Vertex AI Agent Engine Overview

### What is Agent Engine?

**Vertex AI Agent Engine** is Google Cloud's managed runtime for deploying and scaling AI agents in production. It provides:

- **Managed Infrastructure**: No servers to maintain
- **Auto-scaling**: Handle variable traffic automatically
- **Enterprise Security**: VPC-SC, IAM, audit logging
- **Built-in Services**: Sessions, memory, evaluation, observability

### Agent Engine Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        Vertex AI Agent Engine                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │                         Agent Runtime                            │    │
│  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐              │    │
│  │  │   Agent 1   │  │   Agent 2   │  │   Agent 3   │              │    │
│  │  │  (ADK/LG)   │  │  (ADK/LG)   │  │  (ADK/LG)   │              │    │
│  │  └─────────────┘  └─────────────┘  └─────────────┘              │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                                                                          │
│  ┌───────────────────────────────────────────────────────────────────┐  │
│  │                       Platform Services                            │  │
│  │                                                                    │  │
│  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐             │  │
│  │  │   Sessions   │  │    Memory    │  │  Evaluation  │             │  │
│  │  │   Service    │  │    Bank      │  │   Service    │             │  │
│  │  └──────────────┘  └──────────────┘  └──────────────┘             │  │
│  │                                                                    │  │
│  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐             │  │
│  │  │    Code      │  │   Example    │  │ Observability│             │  │
│  │  │  Execution   │  │    Store     │  │   (Tracing)  │             │  │
│  │  └──────────────┘  └──────────────┘  └──────────────┘             │  │
│  └───────────────────────────────────────────────────────────────────┘  │
│                                                                          │
│  ┌───────────────────────────────────────────────────────────────────┐  │
│  │                         Security Layer                             │  │
│  │  • VPC-SC  • IAM  • Audit Logs  • Data Encryption                 │  │
│  └───────────────────────────────────────────────────────────────────┘  │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Platform Services

| Service | Purpose |
|---------|--------|
| **Sessions** | Persist conversation state across interactions |
| **Memory Bank** | Long-term storage for user preferences and facts |
| **Evaluation** | Test agent quality with benchmark datasets |
| **Code Execution** | Safe sandbox for running agent-generated code |
| **Example Store** | Few-shot examples to improve agent responses |
| **Observability** | Tracing, logging, and monitoring |

In [ ]:
# Setup
import os
from getpass import getpass

if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

# For Vertex AI, we also need project configuration
# Uncomment and set these for actual deployment

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'your-project-id'
# os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

print("✅ Setup complete!")
print("\n📝 Note: Actual deployment requires a Google Cloud project with billing enabled.")

## 6.2 Deployment Methods

There are two main ways to deploy agents to Vertex AI:

### Method 1: From Agent Object (Interactive)
- Best for development and testing
- Deploy directly from Python code
- Quick iteration

### Method 2: From Source Files (CI/CD)
- Best for production
- Works with version control
- Integrates with Cloud Build

In [ ]:
# Method 1: Deploy from Agent Object

deploy_object_template = '''
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.cloud import aiplatform
from vertexai import agent_engines

# Initialize Vertex AI
aiplatform.init(
    project="your-project-id",
    location="us-central1"
)

# Define your agent
def get_weather(city: str) -> dict:
    """Get weather for a city."""
    return {"city": city, "temp": 72, "condition": "sunny"}

my_agent = Agent(
    name="weather_agent",
    model="gemini-2.0-flash",
    description="A weather assistant",
    instruction="Help users with weather queries.",
    tools=[FunctionTool(get_weather)]
)

# Deploy to Agent Engine
deployed_agent = agent_engines.create(
    agent=my_agent,
    display_name="Weather Agent",
    description="Production weather assistant"
)

print(f"Agent deployed: {deployed_agent.resource_name}")
'''

print("📝 Method 1: Deploy from Agent Object")
print("=" * 60)
print(deploy_object_template)

In [ ]:
# Method 2: Deploy from Source Files

deploy_source_template = '''
# Project structure:
# my_agent/
# ├── agent.py          # Agent definition
# ├── requirements.txt  # Dependencies
# └── agent.json        # Optional: Agent Card for A2A

# my_agent/agent.py
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

def get_weather(city: str) -> dict:
    """Get weather for a city."""
    return {"city": city, "temp": 72, "condition": "sunny"}

# The agent must be named `root_agent`
root_agent = Agent(
    name="weather_agent",
    model="gemini-2.0-flash",
    description="A weather assistant",
    instruction="Help users with weather queries.",
    tools=[FunctionTool(get_weather)]
)

# my_agent/requirements.txt
google-adk>=1.0.0
google-cloud-aiplatform>=1.70.0

# Deploy via CLI or Cloud Build:
# gcloud agent-engines deploy my_agent/ --display-name="Weather Agent"
'''

print("📝 Method 2: Deploy from Source Files")
print("=" * 60)
print(deploy_source_template)

## 6.3 Production Configuration

### Sessions Service

Sessions maintain conversation state across interactions:

In [ ]:
# Sessions configuration

sessions_example = '''
from vertexai import agent_engines

# Get your deployed agent
agent = agent_engines.get("projects/PROJECT/locations/LOCATION/agents/AGENT_ID")

# Create a session
session = agent.create_session(
    user_id="user_123",
    session_id="session_abc",  # Optional: auto-generated if not provided
    metadata={
        "channel": "web",
        "locale": "en-US"
    }
)

# Send messages in the session
response1 = agent.run(
    session_id=session.id,
    message="What's the weather in Tokyo?"
)

# Continue the conversation
response2 = agent.run(
    session_id=session.id,
    message="What about tomorrow?"  # Agent remembers context!
)

# List all sessions for a user
sessions = agent.list_sessions(user_id="user_123")

# Session state can be accessed
print(f"Session state: {session.state}")
print(f"Message count: {len(session.messages)}")
'''

print("📝 Sessions Service Example:")
print("=" * 60)
print(sessions_example)

In [ ]:
# Memory Bank configuration

memory_example = '''
from google.adk.memory import MemoryService
from vertexai import agent_engines

# Memory Bank stores long-term information about users

# During agent execution, memories are automatically:
# 1. Retrieved based on relevance to current query
# 2. Injected into the agent's context
# 3. Updated when the agent learns new facts

# Configure memory in your agent
from google.adk.agents import Agent
from google.adk.memory import VertexMemoryService

memory_service = VertexMemoryService(
    project="your-project",
    location="us-central1"
)

agent_with_memory = Agent(
    name="personalized_assistant",
    model="gemini-2.0-flash",
    instruction="""
    You are a personalized assistant. Use memories about the user
    to provide tailored responses. Remember their preferences.
    """,
    memory_service=memory_service
)

# Memories are stored as semantic facts:
# - "User prefers vegetarian food"
# - "User lives in San Francisco"
# - "User is learning Python"
'''

print("📝 Memory Bank Example:")
print("=" * 60)
print(memory_example)

## 6.4 Cloud Run Deployment for A2A

For A2A-enabled agents, Cloud Run provides more flexibility:

In [ ]:
# Cloud Run deployment with A2A

cloud_run_template = '''
# Dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# Expose the A2A port
EXPOSE 8080

# Run the A2A server
CMD ["python", "-m", "uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

# main.py
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.a2a import to_a2a
import os

def get_product_info(product_id: str) -> dict:
    """Get product information."""
    return {
        "id": product_id,
        "name": "Sample Product",
        "price": 99.99,
        "in_stock": True
    }

root_agent = Agent(
    name="product_agent",
    model="gemini-2.0-flash",
    description="Product information service",
    instruction="Help with product queries.",
    tools=[FunctionTool(get_product_info)]
)

# Expose as A2A server
app = to_a2a(root_agent)

# Deploy command:
# gcloud run deploy product-agent \\
#   --source . \\
#   --region us-central1 \\
#   --allow-unauthenticated  # Or configure IAM
'''

print("📝 Cloud Run A2A Deployment:")
print("=" * 60)
print(cloud_run_template)

## 6.5 CI/CD with Cloud Build

Production deployments should use CI/CD pipelines:

In [ ]:
# Cloud Build configuration

cloudbuild_yaml = '''
# cloudbuild.yaml
steps:
  # Step 1: Run tests
  - name: "python:3.11"
    entrypoint: "bash"
    args:
      - "-c"
      - |
        pip install -r requirements.txt
        pip install pytest
        pytest tests/ -v

  # Step 2: Run agent evaluation
  - name: "python:3.11"
    entrypoint: "bash"
    args:
      - "-c"
      - |
        pip install -r requirements.txt
        python evaluate_agent.py --threshold 0.8

  # Step 3: Build container
  - name: "gcr.io/cloud-builders/docker"
    args:
      - "build"
      - "-t"
      - "gcr.io/$PROJECT_ID/my-agent:$COMMIT_SHA"
      - "."

  # Step 4: Push to Container Registry
  - name: "gcr.io/cloud-builders/docker"
    args:
      - "push"
      - "gcr.io/$PROJECT_ID/my-agent:$COMMIT_SHA"

  # Step 5: Deploy to Cloud Run
  - name: "gcr.io/cloud-builders/gcloud"
    args:
      - "run"
      - "deploy"
      - "my-agent"
      - "--image"
      - "gcr.io/$PROJECT_ID/my-agent:$COMMIT_SHA"
      - "--region"
      - "us-central1"
      - "--platform"
      - "managed"

# Optional: Add staging/production environments
substitutions:
  _ENVIRONMENT: "staging"

# Trigger on push to main branch
trigger:
  branch: "main"
'''

print("📝 Cloud Build CI/CD Configuration:")
print("=" * 60)
print(cloudbuild_yaml)

## 6.6 Agent Evaluation

Before deploying, evaluate your agent's quality:

In [ ]:
# Agent evaluation example

evaluation_template = '''
# evaluate_agent.py
from vertexai import agent_engines
from vertexai.evaluation import AgentEvaluator
import json

# Define test cases
test_cases = [
    {
        "input": "What's the weather in New York?",
        "expected_tool_calls": ["get_weather"],
        "expected_contains": ["New York", "temperature"]
    },
    {
        "input": "Tell me about tomorrow's forecast in London",
        "expected_tool_calls": ["get_forecast"],
        "expected_contains": ["London", "tomorrow"]
    },
    {
        "input": "What cities can you check weather for?",
        "expected_tool_calls": [],  # Should answer from knowledge
        "expected_contains": ["cities"]
    }
]

# Create evaluator
evaluator = AgentEvaluator(
    agent=my_agent,
    metrics=[
        "tool_call_accuracy",
        "response_relevance",
        "latency_p95",
        "safety_score"
    ]
)

# Run evaluation
results = evaluator.evaluate(test_cases)

# Check results
print(f"Tool Call Accuracy: {results.tool_call_accuracy:.2%}")
print(f"Response Relevance: {results.response_relevance:.2%}")
print(f"P95 Latency: {results.latency_p95:.2f}s")
print(f"Safety Score: {results.safety_score:.2%}")

# Fail if below threshold
if results.overall_score < 0.8:
    raise ValueError(f"Agent quality below threshold: {results.overall_score:.2%}")
'''

print("📝 Agent Evaluation Script:")
print("=" * 60)
print(evaluation_template)

## 6.7 Observability and Monitoring

Production agents need proper monitoring:

In [ ]:
# Observability configuration

observability_template = '''
# Enable Cloud Trace for agent interactions
from opentelemetry import trace
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Set up tracing
tracer_provider = TracerProvider()
cloud_trace_exporter = CloudTraceSpanExporter()
tracer_provider.add_span_processor(
    BatchSpanProcessor(cloud_trace_exporter)
)
trace.set_tracer_provider(tracer_provider)

# ADK automatically instruments agent calls
# You'll see traces for:
# - Agent invocations
# - Tool calls
# - LLM requests
# - A2A communications
# - MCP operations

# Custom metrics
from google.cloud import monitoring_v3

client = monitoring_v3.MetricServiceClient()
project_name = f"projects/{PROJECT_ID}"

# Create custom metrics for:
# - Agent response time
# - Tool success rate
# - User satisfaction score
# - Error rate by type

# Set up alerts
# - High error rate > 5%
# - P95 latency > 10s
# - Memory usage > 80%
'''

print("📝 Observability Configuration:")
print("=" * 60)
print(observability_template)

## 6.8 Security Best Practices

### Production Security Checklist

```
┌─────────────────────────────────────────────────────────────────┐
│                    Security Checklist                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Authentication & Authorization                                  │
│  ☐ IAM roles configured (least privilege)                       │
│  ☐ Service accounts with minimal permissions                    │
│  ☐ API keys rotated regularly                                   │
│  ☐ OAuth/OIDC for user authentication                          │
│                                                                  │
│  Network Security                                                │
│  ☐ VPC Service Controls enabled                                 │
│  ☐ Private Google Access configured                             │
│  ☐ Cloud Armor for DDoS protection                              │
│  ☐ SSL/TLS certificates valid                                   │
│                                                                  │
│  Data Protection                                                 │
│  ☐ Customer-managed encryption keys (CMEK)                      │
│  ☐ Data residency requirements met                              │
│  ☐ PII handling policies enforced                               │
│  ☐ Audit logging enabled                                        │
│                                                                  │
│  Agent Safety                                                    │
│  ☐ Input validation and sanitization                            │
│  ☐ Output content filtering                                     │
│  ☐ Rate limiting configured                                     │
│  ☐ Tool permissions scoped appropriately                        │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Security configuration example

security_template = '''
# Secure agent configuration
from google.adk.agents import Agent
from google.adk.security import (
    InputValidator,
    OutputFilter,
    RateLimiter,
    ToolPermissions
)

# Input validation
input_validator = InputValidator(
    max_length=10000,
    block_patterns=["<script>", "javascript:"],
    sanitize_html=True
)

# Output filtering  
output_filter = OutputFilter(
    block_pii=True,  # Detect and block PII in responses
    block_profanity=True,
    max_response_length=50000
)

# Rate limiting
rate_limiter = RateLimiter(
    requests_per_minute=60,
    requests_per_day=1000,
    by_user=True
)

# Tool permissions (principle of least privilege)
tool_permissions = ToolPermissions(
    allowed_domains=["api.weather.com", "internal.company.com"],
    blocked_operations=["delete", "drop"],
    max_data_access_rows=1000
)

# Secure agent
secure_agent = Agent(
    name="secure_assistant",
    model="gemini-2.0-flash",
    instruction="...",
    input_validator=input_validator,
    output_filter=output_filter,
    rate_limiter=rate_limiter,
    tool_permissions=tool_permissions
)
'''

print("📝 Security Configuration:")
print("=" * 60)
print(security_template)

## 6.9 Agent Starter Pack

Google provides the **Agent Starter Pack** for production deployments:

```bash
# Clone the starter pack
git clone https://github.com/GoogleCloudPlatform/agent-starter-pack
cd agent-starter-pack

# Initialize for your project
./init.sh --project=YOUR_PROJECT --region=us-central1

# This sets up:
# - Terraform infrastructure
# - Cloud Build CI/CD
# - Monitoring dashboards
# - Security configurations
# - Example agents
```

### What's Included

| Component | Description |
|-----------|-------------|
| **Terraform** | Infrastructure as Code for all GCP resources |
| **Cloud Build** | CI/CD pipelines with testing and evaluation |
| **Observability** | Pre-configured dashboards and alerts |
| **Templates** | Production-ready agent templates |
| **Best Practices** | Security, scaling, and reliability patterns |

## 6.10 Hands-On: Prepare for Deployment

Let's create the structure for a production-ready agent:

In [ ]:
# Production agent structure

production_structure = '''
my-production-agent/
├── agent/
│   ├── __init__.py
│   ├── agent.py           # Main agent definition (root_agent)
│   ├── tools.py           # Tool definitions
│   └── prompts.py         # Instruction templates
│
├── tests/
│   ├── __init__.py
│   ├── test_agent.py      # Unit tests
│   ├── test_tools.py      # Tool tests
│   └── test_integration.py # Integration tests
│
├── evaluation/
│   ├── test_cases.json    # Evaluation test cases
│   └── evaluate.py        # Evaluation script
│
├── infrastructure/
│   ├── main.tf            # Terraform configuration
│   ├── variables.tf       # Terraform variables
│   └── outputs.tf         # Terraform outputs
│
├── .github/
│   └── workflows/
│       └── deploy.yaml    # GitHub Actions (alternative to Cloud Build)
│
├── Dockerfile             # Container definition
├── cloudbuild.yaml        # Cloud Build configuration
├── requirements.txt       # Python dependencies
├── agent.json             # A2A Agent Card
└── README.md              # Documentation
'''

print("📁 Production Agent Structure:")
print("=" * 60)
print(production_structure)

In [ ]:
# Create the agent.py file content

agent_py_content = '''
"""Production agent definition."""

import os
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .tools import (
    search_products,
    get_product_details,
    check_inventory,
    create_order
)
from .prompts import AGENT_INSTRUCTION

# Environment configuration
MODEL = os.getenv("AGENT_MODEL", "gemini-2.0-flash")
DEBUG = os.getenv("AGENT_DEBUG", "false").lower() == "true"

# Define the root agent (required name for deployment)
root_agent = Agent(
    name="product_assistant",
    model=MODEL,
    description="A product assistant that helps customers find and order products",
    instruction=AGENT_INSTRUCTION,
    tools=[
        FunctionTool(search_products),
        FunctionTool(get_product_details),
        FunctionTool(check_inventory),
        FunctionTool(create_order)
    ]
)

# For local development/testing
if __name__ == "__main__":
    from google.adk.runners import InMemoryRunner
    import asyncio
    
    async def test():
        runner = InMemoryRunner(agent=root_agent)
        # Test interactions here
        print("Agent ready for testing")
    
    asyncio.run(test())
'''

print("📝 agent/agent.py:")
print("=" * 60)
print(agent_py_content)

---

## Summary

In this module, you learned:

1. **Agent Engine Architecture**: Managed runtime with built-in services
2. **Deployment Methods**: From object (interactive) vs from source (CI/CD)
3. **Production Services**: Sessions, Memory Bank, Evaluation
4. **Cloud Run**: Deploying A2A-enabled agents
5. **CI/CD**: Cloud Build pipelines for agent deployment
6. **Security**: Best practices for production agents

### Key Takeaways

- Agent Engine provides managed infrastructure for production
- Use Sessions for conversation continuity
- Use Memory Bank for personalization
- Always evaluate agents before deployment
- Follow security best practices

### Next Steps

Continue to **Module 7: Best Practices and Capstone** to learn advanced patterns and work on your capstone project.

---

### Resources

- [Vertex AI Agent Engine Documentation](https://cloud.google.com/vertex-ai/docs/agent-builder/agent-engine)
- [Agent Starter Pack](https://github.com/GoogleCloudPlatform/agent-starter-pack)
- [ADK Deployment Guide](https://google.github.io/adk-docs/deploy/)
- [Cloud Run Documentation](https://cloud.google.com/run/docs)